In [40]:
import numpy as np
import qiskit
%matplotlib inline

In [41]:
from qiskit import BasicAer
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import Pauli, pauli_group
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.operators.primitive_ops import PauliOp
from qiskit.aqua.operators.list_ops import SummedOp
from qiskit.aqua.algorithms import NumPyMinimumEigensolver, VQE
from qiskit.aqua.components.optimizers import COBYLA

Generate a random 4x4 Hamiltonian

In [42]:
scale = 10
H_weight = scale*np.random.random(16)

In [43]:
paulis = pauli_group(2)
pauli_operators = []
for i in range(16):
    pauli_operators.append(PauliOp(paulis[i], H_weight[i]))
H_operator = SummedOp(pauli_operators)

In [44]:
H_operator.to_matrix()

array([[12.18537168 +0.j        ,  0.75728355-13.10833852j,
        10.05994718-15.27009852j, -2.37613763-13.33994015j],
       [ 0.75728355+13.10833852j,  6.26269075 +0.j        ,
        11.78751369 -4.90462467j,  9.93854364 -0.76911875j],
       [10.05994718+15.27009852j, 11.78751369 +4.90462467j,
         4.38263651 +0.j        ,  0.02705532 -3.7878785j ],
       [-2.37613763+13.33994015j,  9.93854364 +0.76911875j,
         0.02705532 +3.7878785j ,  9.87721702 +0.j        ]])

Using classical algorithm to calculate the minimum energy

In [45]:
classical_algo = NumPyMinimumEigensolver(H_operator.to_pauli_op())
classical_result = classical_algo.run()
np.real(classical_result.eigenvalue)

-16.25508011615957

Create parametrized circuit to generate the trial state

In [46]:
circuit = TwoLocal(num_qubits=2, rotation_blocks= ['rz','ry','rz'],
                   entanglement_blocks='cx', entanglement=[[(0,1)],[(1,0)],[(0,1)]], reps = 3)

In [47]:
print(circuit)

     ┌──────────┐┌──────────┐┌──────────┐     ┌──────────┐┌──────────┐»
q_0: ┤ RZ(θ[0]) ├┤ RY(θ[2]) ├┤ RZ(θ[4]) ├──■──┤ RZ(θ[6]) ├┤ RY(θ[8]) ├»
     ├──────────┤├──────────┤├──────────┤┌─┴─┐├──────────┤├──────────┤»
q_1: ┤ RZ(θ[1]) ├┤ RY(θ[3]) ├┤ RZ(θ[5]) ├┤ X ├┤ RZ(θ[7]) ├┤ RY(θ[9]) ├»
     └──────────┘└──────────┘└──────────┘└───┘└──────────┘└──────────┘»
«     ┌───────────┐┌───┐┌───────────┐┌───────────┐┌───────────┐     »
«q_0: ┤ RZ(θ[10]) ├┤ X ├┤ RZ(θ[12]) ├┤ RY(θ[14]) ├┤ RZ(θ[16]) ├──■──»
«     ├───────────┤└─┬─┘├───────────┤├───────────┤├───────────┤┌─┴─┐»
«q_1: ┤ RZ(θ[11]) ├──■──┤ RZ(θ[13]) ├┤ RY(θ[15]) ├┤ RZ(θ[17]) ├┤ X ├»
«     └───────────┘     └───────────┘└───────────┘└───────────┘└───┘»
«     ┌───────────┐┌───────────┐┌───────────┐
«q_0: ┤ RZ(θ[18]) ├┤ RY(θ[20]) ├┤ RZ(θ[22]) ├
«     ├───────────┤├───────────┤├───────────┤
«q_1: ┤ RZ(θ[19]) ├┤ RY(θ[21]) ├┤ RZ(θ[23]) ├
«     └───────────┘└───────────┘└───────────┘


Running the VQE algorithm

In [48]:
optimizer = COBYLA(maxiter=10000)
algo = VQE(H_operator, circuit, optimizer)
result = algo.run(QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                seed_simulator=aqua_globals.random_seed,
                                seed_transpiler=aqua_globals.random_seed))

In [49]:
print("The algorithm run time: {}".format(result.optimizer_time))
print("The minimum energy found by VQE: {}".format(result.optimal_value))
print("The minimum energy found by classical algorithm: {}".format(np.real(classical_result.eigenvalue)))

The algorithm run time: 36.87755537033081
The minimum energy found by VQE: -16.25508005441619
The minimum energy found by classical algorithm: -16.25508011615957
